In [ ]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')


Connected to MongoDB


In [1]:
employee_data = [
    {'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500},
    {'name': 'Bob', 'department': 'Finance', 'age': 35, 'salary': 5200},
    {'name': 'Charlie', 'department': 'IT', 'age': 27, 'salary': 6000},
    {'name': 'Diana', 'department': 'IT', 'age': 31, 'salary': 7500},
    {'name': 'Evan', 'department': 'HR', 'age': 26, 'salary': 4000},
    {'name': 'Fiona', 'department': 'HR', 'age': 30, 'salary': 4300},
    {'name': 'George', 'department': 'Marketing', 'age': 33, 'salary': 5500},
    {'name': 'Hannah', 'department': 'Marketing', 'age': 28, 'salary': 4800},
    {'name': 'Ian', 'department': 'Sales', 'age': 32, 'salary': 6700},
    {'name': 'Julia', 'department': 'Sales', 'age': 25, 'salary': 6200}
]
collection.create_index(
    [('department', 1), ('name', 1)],
    unique=True
)


for new_employee in employee_data:
    if collection.find_one({'name': new_employee['name'], 'department': new_employee['department']}):
        print(f"Data untuk {new_employee['name']} di {new_employee['department']} sudah ada, tidak ditambahkan.")
    else:
        collection.insert_one(new_employee)
        print(f"Data baru untuk {new_employee['name']} berhasil ditambahkan.")

for employee in collection.find():
    print(employee)

NameError: name 'collection' is not defined

In [ ]:
# Query Aggregation untuk mencari rata-rata gaji per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]
for result in collection.aggregate(pipeline):
    print(result)


{'_id': 'IT', 'average_salary': 6750.0}
{'_id': 'Marketing', 'average_salary': 5150.0}
{'_id': 'HR', 'average_salary': 4150.0}
{'_id': 'Sales', 'average_salary': 6450.0}
{'_id': 'Finance', 'average_salary': 4850.0}


In [ ]:
pipeline_top5 = [
    {'$sort': {'department': 1, 'salary': -1}},
    {'$group': {
        '_id': '$department',
        'topEmployees': {'$push': {'name': '$name', 'salary': '$salary'}}
    }},
    {'$project': {
        'department': '$_id',
        'top5': {'$slice': ['$topEmployees', 5]},
        '_id': 0
    }}
]

for result in collection.aggregate(pipeline_top5):
    print(result)

{'department': 'Finance', 'top5': [{'name': 'Bob', 'salary': 5200}, {'name': 'Bob', 'salary': 5200}, {'name': 'Alice', 'salary': 4500}, {'name': 'Alice', 'salary': 4500}]}
{'department': 'Marketing', 'top5': [{'name': 'George', 'salary': 5500}, {'name': 'George', 'salary': 5500}, {'name': 'Hannah', 'salary': 4800}, {'name': 'Hannah', 'salary': 4800}]}
{'department': 'HR', 'top5': [{'name': 'Fiona', 'salary': 4300}, {'name': 'Fiona', 'salary': 4300}, {'name': 'Evan', 'salary': 4000}, {'name': 'Evan', 'salary': 4000}]}
{'department': 'Sales', 'top5': [{'name': 'Ian', 'salary': 6700}, {'name': 'Ian', 'salary': 6700}, {'name': 'Julia', 'salary': 6200}, {'name': 'Julia', 'salary': 6200}]}
{'department': 'IT', 'top5': [{'name': 'Diana', 'salary': 7500}, {'name': 'Diana', 'salary': 7500}, {'name': 'Charlie', 'salary': 6000}, {'name': 'Charlie', 'salary': 6000}]}


In [ ]:
result = collection.delete_many({'age': {'$lt': 25}})
print(f"{result.deleted_count} karyawan dengan usia di bawah 25 tahun telah dihapus.")


0 karyawan dengan usia di bawah 25 tahun telah dihapus.


In [2]:
pipeline_report = [
    {'$group': {
        '_id': '$department',
        'total_salary': {'$sum': '$salary'},
        'avg_age': {'$avg': '$age'},
        'avg_salary': {'$avg': '$salary'},
        'count': {'$sum': 1}
    }},
    {'$sort': {'total_salary': -1}}
]

for result in collection.aggregate(pipeline_report):
    print(result)

NameError: name 'collection' is not defined

In [ ]:
collection_products = db['products']
products_data = [
    {'name': 'Laptop X', 'category': 'Electronics', 'price': 15000, 'stock': 25},
    {'name': 'Smartphone Y', 'category': 'Electronics', 'price': 8000, 'stock': 50},
    {'name': 'Headphones Z', 'category': 'Accessories', 'price': 1200, 'stock': 80},
    {'name': 'Office Chair', 'category': 'Furniture', 'price': 3500, 'stock': 40},
    {'name': 'Desk Lamp', 'category': 'Furniture', 'price': 900, 'stock': 100},
    {'name': 'Water Bottle', 'category': 'Home', 'price': 300, 'stock': 150},
    {'name': 'Notebook', 'category': 'Stationery', 'price': 50, 'stock': 500},
    {'name': 'Mechanical Keyboard', 'category': 'Electronics', 'price': 2200, 'stock': 60},
    {'name': 'Mouse', 'category': 'Accessories', 'price': 600, 'stock': 90},
    {'name': 'Coffee Mug', 'category': 'Home', 'price': 400, 'stock': 120}
]

collection_products.create_index(
    [('category', 1), ('name', 1)],
    unique=True
)

try:
    collection_products.insert_many(product_data, ordered=False)
    print("Semua data produk berhasil dimasukkan.")
except errors.BulkWriteError as e:
    print("Beberapa data sudah ada dan dilewati (duplikasi terdeteksi).")

for product in collection_products.find():
    print(product)

10 produk berhasil dimasukkan ke koleksi 'products'.
{'_id': ObjectId('69033f59bface3c998bccec1'), 'name': 'Laptop X', 'category': 'Electronics', 'price': 15000, 'stock': 25}
{'_id': ObjectId('69033f59bface3c998bccec2'), 'name': 'Smartphone Y', 'category': 'Electronics', 'price': 8000, 'stock': 50}
{'_id': ObjectId('69033f59bface3c998bccec3'), 'name': 'Headphones Z', 'category': 'Accessories', 'price': 1200, 'stock': 80}
{'_id': ObjectId('69033f59bface3c998bccec4'), 'name': 'Office Chair', 'category': 'Furniture', 'price': 3500, 'stock': 40}
{'_id': ObjectId('69033f59bface3c998bccec5'), 'name': 'Desk Lamp', 'category': 'Furniture', 'price': 900, 'stock': 100}
{'_id': ObjectId('69033f59bface3c998bccec6'), 'name': 'Water Bottle', 'category': 'Home', 'price': 300, 'stock': 150}
{'_id': ObjectId('69033f59bface3c998bccec7'), 'name': 'Notebook', 'category': 'Stationery', 'price': 50, 'stock': 500}
{'_id': ObjectId('69033f59bface3c998bccec8'), 'name': 'Mechanical Keyboard', 'category': 'Elect

In [ ]:
avg_price = collection_products.aggregate([
    {'$group': {'_id': None, 'avg_price': {'$avg': '$price'}}}
])
avg_price_value = list(avg_price)[0]['avg_price']

print(f"Rata-rata harga produk: {avg_price_value}")
print("Produk dengan harga di atas rata-rata:")

for product in collection_products.find({'price': {'$gt': avg_price_value}}):
    print(product)

Rata-rata harga produk: 3215.0
Produk dengan harga di atas rata-rata:
{'_id': ObjectId('69033b29bface3c998bcce8e'), 'name': 'Laptop X', 'category': 'Electronics', 'price': 15000, 'stock': 25}
{'_id': ObjectId('69033b29bface3c998bcce8f'), 'name': 'Smartphone Y', 'category': 'Electronics', 'price': 8000, 'stock': 50}
{'_id': ObjectId('69033b29bface3c998bcce91'), 'name': 'Office Chair', 'category': 'Furniture', 'price': 3500, 'stock': 40}
{'_id': ObjectId('69033b5ebface3c998bcce98'), 'name': 'Laptop X', 'category': 'Electronics', 'price': 15000, 'stock': 25}
{'_id': ObjectId('69033b5ebface3c998bcce99'), 'name': 'Smartphone Y', 'category': 'Electronics', 'price': 8000, 'stock': 50}
{'_id': ObjectId('69033b5ebface3c998bcce9b'), 'name': 'Office Chair', 'category': 'Furniture', 'price': 3500, 'stock': 40}
{'_id': ObjectId('69033bd9bface3c998bccead'), 'name': 'Laptop X', 'category': 'Electronics', 'price': 15000, 'stock': 25}
{'_id': ObjectId('69033bd9bface3c998bcceae'), 'name': 'Smartphone Y'

In [ ]:
pipeline = [
    {'$group': {'_id': '$category', 'total_products': {'$sum': 1}}}
]

for result in collection_products.aggregate(pipeline):
    print(result)

{'_id': 'Furniture', 'total_products': 6}
{'_id': 'Accessories', 'total_products': 6}
{'_id': 'Electronics', 'total_products': 9}
{'_id': 'Home', 'total_products': 6}
{'_id': 'Stationery', 'total_products': 3}
